In [ ]:
using CSV
using DataFrames
using JLD2
using FileIO
using Plots
using StatsPlots
using StatsBase
using Statistics

In [ ]:
include("optimization_Util.jl")

### Visulization of the evaluated models

Load evaluation data

In [ ]:
file = File(format"JLD2", "Data/addap_output_1.jld2")
addap_output_1 = JLD2.load(file)
file = File(format"JLD2", "Data/addap_output_0,1.jld2")
addap_output_01 = JLD2.load(file)
file = File(format"JLD2", "Data/multi_output_1.jld2")
multi_output_1 = JLD2.load(file)
file = File(format"JLD2", "Data/multi_output_0,1.jld2")
multi_output_01 = JLD2.load(file)
file = File(format"JLD2", "Data/regulized_output.jld2")
regulized_output = JLD2.load(file)
file = File(format"JLD2", "Data/true_output.jld2")
true_output = JLD2.load(file)

save TSP evaluations

In [ ]:
df = DataFrame(Addap_1 = [x[2] for x in addap_output_1["solutions"]],
                Addap_01 = [x[2] for x in addap_output_01["solutions"]],
                Multi_1 = [x[2] for x in multi_output_1["solutions"]],
                Multi_01 = [x[2] for x in multi_output_01["solutions"]],
                Generic_regu = [x[2] for x in regulized_output["solutions"]],
                true_solutions = [x[2] for x in true_output["solutions"]]);

save loss 

In [ ]:
Addap_1 = [x for x in addap_output_1["trainings_losses"]]
Addap_01 = [x for x in addap_output_01["trainings_losses"]]
Multi_1 = [x for x in multi_output_1["trainings_losses"]]
Multi_01 = [x for x in multi_output_01["trainings_losses"]]
Generic_regu = [x for x in regulized_output["trainings_losses"]]

In [ ]:
df_losses = DataFrame(Addap_1 = Addap_1,
                     Addap_01 = Addap_01,
                     Multi_1 = vcat(Multi_1, fill(missing, 200 - length(Multi_1))),
                     Multi_01 = vcat(Multi_01, fill(missing, 200 - length(Multi_01))),
                     Generic_regu = vcat(Generic_regu, fill(missing, 200 - length(Generic_regu))));

Build plots of losses and save them as png

In [ ]:
plot(skipmissing(df_losses[!, :Addap_1]),  title="Addap 1", xlabel = "Epoch", legend = false)
savefig("graphs/Addap_1.png")  


In [ ]:
plot(skipmissing(df_losses[!, :Addap_01]),  title="Addap 1", xlabel = "Epoch", legend = false)
savefig("graphs/Addap_01.png")  

In [ ]:
plot3 = plot(skipmissing(df_losses[!, :Generic_regu]), title="Generic regu",  xlabel = "Epoch",legend = false)
savefig("graphs/Generic_regu.png")  

In [ ]:
plot3 = plot(skipmissing(df_losses[!, :Multi_01]), title="Multi 0.1", xlabel = "Epoch", legend = false)
savefig("graphs/Multi_01.png")  

In [ ]:
plot3 = plot(skipmissing(df_losses[!, :Multi_1]), title="Multi 1",  xlabel = "Epoch",legend = false)
savefig("graphs/Multi_1.png")  

Normalize TSP evaluation metric as % of true solution

In [ ]:
df[!, :Addap_1] = df[!, :Addap_1] ./ df[!, :true_solutions]
df[!, :Addap_01] = df[!, :Addap_01] ./ df[!, :true_solutions]
df[!, :Multi_1] = df[!, :Multi_1] ./ df[!, :true_solutions]
df[!, :Multi_01] = df[!, :Multi_01] ./ df[!, :true_solutions]
df[!, :Generic_regu] = df[!, :Generic_regu] ./ df[!, :true_solutions]

Stack in Dataframe and rename Columns

In [ ]:
df_long = stack(df, [:Addap_1, :Addap_01, :Multi_1, :Multi_01, :Generic_regu])
replace!(df_long.variable, "Addap_01" => "Addap_0.1")
replace!(df_long.variable, "Multi_01" => "Multi_0.1")

Draw boxplot

In [ ]:
@df df_long boxplot(string.(:variable), :value, legend = false)
ylabel!("relative distance to true assignments")

In [ ]:
savefig("boxplot.png")  